# Procesamiento de lenguaje natural Supermercados JSON

Ahora que tenemos lista la base de datos lista -> empezaremos a crear una solución para poder crear una separación en gramajes cuando exsitan en los titulos de productos.
Además con ayuda de una tecnica de Procesamiento de Lenguaje Natural para extraer palabras clave los productos y poder clasificarlos de mejor manera.

Primero revisemos que tengamos listas estas librerias instaladas ->

In [48]:
#python3 -m spacy download en_core_web_sm

In [49]:
#!python3 -m spacy download es_core_news_sm

El flujo para este desarrollo sera el siguiente:

1. Crearemos el filtrado mediante regex para obtener el gramaje total, validando si son kilogramos o gramos
2. Normalizaremos los datos una vez más
3. Creamos la solución con Procesamiento de Lenguaje Natural para extraer palabras claves

-----------------------------------

### Iniciemos!

De productosChedraui.json y productosSoriana.json obtener los productos que esten en ambos json

In [50]:
import pandas as pd
import json
import re
import pandas as pd
from pymongo import MongoClient
import spacy
from collections import Counter
from spacy.lang.es import STOP_WORDS

In [51]:
pChedraui = []
with open('productosChedraui.json' , encoding="UTF-8") as f:
    file_data = json.load(f)
    for p in file_data:
        nombre = p['nombre']
        precio = p['precio']
        busqueda = p['busqueda']
        url = p['url']
        tienda = p['tienda']
        fecha = p['fecha']
        producto = {
            "precio": precio,
            "busqueda": busqueda,
            "nombre": nombre,
            "url": url,
            "tienda": tienda,
            "fecha": fecha
        }
        pChedraui.append(producto)


In [52]:
import json
pSoriana = []
with open('productosSoriana.json' , encoding="UTF-8") as f:
    file_data = json.load(f)
    for p in file_data:
        nombre = p['nombre']
        precio = p['precio']
        busqueda = p['busqueda']
        url = p['url']
        tienda = p['tienda']
        fecha = p['fecha']
        producto = {
            "precio": precio,
            "busqueda": busqueda,
            "nombre": nombre,
            "url": url,
            "tienda": tienda,
            "fecha": fecha
        }
        pSoriana.append(producto)


In [53]:
pSoriana[2]

{'precio': 209.0,
 'busqueda': 'Comida para gato',
 'nombre': 'Alimento Para Gato Whiskas 3 Kg',
 'url': 'https://www.soriana.com/alimento-para-gato-whiskas-3-kg/11586931.html',
 'tienda': 'Soriana',
 'fecha': '2023-04-07'}

Separar de nombre la marca y el gramaje con regular expresions


In [54]:
for p in pSoriana:
    nombre = p['nombre']
    gramaje = re.findall(r'(\d+)', nombre)
    if len(gramaje) == 0:
        #el producto no tiene gramaje
        p['gramaje'] = "0"
    if len(gramaje) == 1:
         p['gramaje'] = gramaje[0]     
    if len(gramaje) == 2:
        gramaje = gramaje[0] + "." + gramaje[1]
        p['gramaje'] = gramaje

print(pSoriana)


[{'precio': 71.1, 'busqueda': 'Comida para gato', 'nombre': 'Cepillo Hartz para Gatos', 'url': 'https://www.soriana.com/cepillo-hartz-para-gatos/586451.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': '0'}, {'precio': 122.0, 'busqueda': 'Comida para gato', 'nombre': 'Juguete Rascadero para Gato Zizoo', 'url': 'https://www.soriana.com/juguete-rascadero-para-gato-zizoo/2151902.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': '0'}, {'precio': 209.0, 'busqueda': 'Comida para gato', 'nombre': 'Alimento Para Gato Whiskas 3 Kg', 'url': 'https://www.soriana.com/alimento-para-gato-whiskas-3-kg/11586931.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': '3'}, {'precio': 126.0, 'busqueda': 'Comida para gato', 'nombre': 'Alimento Para Gato Yum Minino 1.5 Kg', 'url': 'https://www.soriana.com/alimento-para-gato-yum-minino-1.5-kg/11485836.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': '1.5'}, {'precio': 103.0, 'busqueda': 'Comida para gato', 'nombre

In [55]:
df = pd.DataFrame(pSoriana)
df

,precio,busqueda,nombre,url,tienda,fecha,gramaje
0,71.1,Comida para gato,Cepillo Hartz para Gatos,https://www.soriana.com/cepillo-hartz-para-gat...,Soriana,2023-04-07,0
1,122.0,Comida para gato,Juguete Rascadero para Gato Zizoo,https://www.soriana.com/juguete-rascadero-para...,Soriana,2023-04-07,0
2,209.0,Comida para gato,Alimento Para Gato Whiskas 3 Kg,https://www.soriana.com/alimento-para-gato-whi...,Soriana,2023-04-07,3
3,126.0,Comida para gato,Alimento Para Gato Yum Minino 1.5 Kg,https://www.soriana.com/alimento-para-gato-yum...,Soriana,2023-04-07,1.5
4,103.0,Comida para gato,Alimento Para Gato Baby Minino 1.3 Kg,https://www.soriana.com/alimento-para-gato-bab...,Soriana,2023-04-07,1.3
5,117.0,Comida para gato,Alimento para gato Whiskas meat mix 1.5kg,https://www.soriana.com/alimento-para-gato-whi...,Soriana,2023-04-07,1.5
6,23.5,Comida para gato,Alimento para gato One cat super nutriente,https://www.soriana.com/alimento-para-gato-one...,Soriana,2023-04-07,0
7,75.0,Comida para gato,Juguete P/Gato Zizoo Raton Natural,https://www.soriana.com/juguete-p-gato-zizoo-r...,Soriana,2023-04-07,0
8,38.5,Comida para gato,Botana Para Gato 90 Gr Atun Zizoo,https://www.soriana.com/botana-para-gato-90-gr...,Soriana,2023-04-07,90
9,40.5,Comida para gato,Alimento Para Gato Animal Planet 45 gr,https://www.soriana.com/alimento-para-gato-ani...,Soriana,2023-04-07,45


In [56]:
for p in pChedraui:
    nombre = p['nombre']
    gramaje = re.findall(r'(\d+)', nombre)
    if len(gramaje) == 0:
        #el producto no tiene gramaje
        marca = nombre
    if len(gramaje) == 1:
         p['gramaje'] = gramaje[0]     
    if len(gramaje) == 2:
        gramaje = gramaje[0] + "." + gramaje[1]
        p['gramaje'] = gramaje

print(pChedraui)

[{'precio': '151.00', 'busqueda': 'Comida para gato', 'nombre': 'Vino Tinto Chileno Gato Negro Red Blend 750ml', 'url': 'https://www.chedraui.com.mx/vino-tinto-chileno-gato-negro-red-blend-750ml-3716120/p', 'tienda': 'Chedraui', 'fecha': '2023-04-07', 'gramaje': '750'}, {'precio': '549.00', 'busqueda': 'Comida para gato', 'nombre': 'Gato HidrÃ¡ulico de Botella Tamer 2 Toneladas', 'url': 'https://www.chedraui.com.mx/gato-hidraulico-de-botella-tamer-2-toneladas-3662401/p', 'tienda': 'Chedraui', 'fecha': '2023-04-07', 'gramaje': '2'}, {'precio': '139.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-04-07', 'gramaje': '1.5'}, {'precio': '120.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg', 'url': 'https://www.chedraui.

Si el gramaje es menor a 50 entonces esta medido en kilogramos , si es mayor a 50 entonces esta medido en gramos -> 1000 gramos = 1 kilogramo , añadir una columna que sea peso en gramos

In [57]:
for x in pChedraui:
    if float(x['gramaje']) < 50:
        x['gramaje'] = float(x['gramaje']) * 1000
    else:
        x['gramaje'] = float(x['gramaje'])

for x in pSoriana:
    if float(x['gramaje']) < 50:
        x['gramaje'] = float(x['gramaje']) * 1000
    else:
        x['gramaje'] = float(x['gramaje'])

Creamos la columna precio por gramo

In [58]:

for x in pChedraui:
    if float(x['gramaje']) == 0.0:
        x['precioPorGramo'] = 0.0
    else:
        x['precioPorGramo'] = float(x['precio']) / float(x['gramaje'])

for x in pSoriana:
    if float(x['gramaje']) == 0.0:
        print(x)
        x['precioPorGramo'] = 0.0
    else:
        x['precioPorGramo'] = float(x['precio']) / float(x['gramaje'])

{'precio': 71.1, 'busqueda': 'Comida para gato', 'nombre': 'Cepillo Hartz para Gatos', 'url': 'https://www.soriana.com/cepillo-hartz-para-gatos/586451.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': 0.0}
{'precio': 122.0, 'busqueda': 'Comida para gato', 'nombre': 'Juguete Rascadero para Gato Zizoo', 'url': 'https://www.soriana.com/juguete-rascadero-para-gato-zizoo/2151902.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': 0.0}
{'precio': 23.5, 'busqueda': 'Comida para gato', 'nombre': 'Alimento para gato One  cat super nutriente', 'url': 'https://www.soriana.com/alimento-para-gato-one--cat-super-nutriente/11554920.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': 0.0}
{'precio': 75.0, 'busqueda': 'Comida para gato', 'nombre': 'Juguete P/Gato Zizoo Raton Natural', 'url': 'https://www.soriana.com/juguete-p-gato-zizoo-raton-natural/2156069.html', 'tienda': 'Soriana', 'fecha': '2023-04-07', 'gramaje': 0.0}


In [59]:
#imprimir pChedraui de manera ordenada
df = pd.DataFrame(pChedraui)
df2 = pd.DataFrame(pSoriana)

Quizas utilizemos almacenarlo en grupos dentro de nuestra base datos!

Unimos los dos dataframes

In [60]:
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
result = pd.concat([df, df2], ignore_index=True)

In [61]:
# Agrupar los datos por busqueda
grupos = result.groupby('busqueda')

# Obtener el grupo Comida para gato
comida_gato = grupos.get_group('Comida para gato')
print('Comida para gato:')
print(comida_gato)

# Obtener el grupo Mayonesa
mayonesa = grupos.get_group('Mayonesa')
print('Mayonesa:')
print(mayonesa)

# Obtener el grupo Fabuloso
fabuloso = grupos.get_group('Fabuloso')
print('Fabuloso:')
print(fabuloso)

Comida para gato:
    precio          busqueda  \
0   151.00  Comida para gato   
1   549.00  Comida para gato   
2   139.00  Comida para gato   
3   120.00  Comida para gato   
4   259.00  Comida para gato   
5    14.00  Comida para gato   
6    14.00  Comida para gato   
7    11.80  Comida para gato   
8    11.45  Comida para gato   
9    10.25  Comida para gato   
10   11.45  Comida para gato   
11  103.00  Comida para gato   
27    71.1  Comida para gato   
28   122.0  Comida para gato   
29   209.0  Comida para gato   
30   126.0  Comida para gato   
31   103.0  Comida para gato   
32   117.0  Comida para gato   
33    23.5  Comida para gato   
34    75.0  Comida para gato   
35    38.5  Comida para gato   
36    40.5  Comida para gato   
37   120.0  Comida para gato   
38   120.0  Comida para gato   
39    15.4  Comida para gato   
40   120.0  Comida para gato   
41   155.0  Comida para gato   
42    15.4  Comida para gato   
43   120.0  Comida para gato   

                     

In [62]:
# Conectar con MongoDB
client = MongoClient('localhost', 27017)
db = client['productos']
coleccion = db['grupos']

# Almacenar cada grupo en MongoDB
for nombre_grupo, grupo in grupos:
    grupo_dict = grupo.to_dict(orient='records')
    coleccion.insert_many(grupo_dict)
    print('Grupo', nombre_grupo, 'almacenado en MongoDB.')

Grupo Comida para gato almacenado en MongoDB.
Grupo Fabuloso almacenado en MongoDB.
Grupo Mayonesa almacenado en MongoDB.


## PROCESAMIENTO DE LENGUAJE NATURAL

---------------------------------------

En esta función estamos definiendo la lógica para eliminar la información adicional del producto (en este caso, el peso y la marca) antes de pasar el texto a la función get_keywords. La función convierte el texto a minúsculas, lo divide en palabras y elimina las palabras finales que representan el peso o la marca del producto (si están presentes).

In [ ]:
def preprocess(text):
    # Eliminar la información adicional del producto (peso, marca, etc.)
    text = text.lower()
    text = text.split(" ")
    if len(text) > 1:
        if text[-1].isdigit():
            text = " ".join(text[:-2])
        else:
            text = " ".join(text[:-1])
    else:
        text = text[0]
    return text

En esta función estamos definiendo la lógica para extraer las palabras clave del texto. Primero, llamamos a la función preprocess para eliminar la información adicional del producto. Luego, procesamos el texto con el objeto nlp de SpaCy y obtenemos las frases y los tokens relevantes para TextRank (sustantivos, adjetivos, verbos y adverbios). A continuación, calculamos el score de TextRank para cada palabra y ordenamos las palabras por score. Finalmente, seleccionamos las 10 palabras con el score más alto y las filtramos para eliminar las palabras vacías y las frases.

In [63]:
nlp = spacy.load("es_core_news_sm")
def get_keywords(text):
    text = preprocess(text)
    doc = nlp(text)
    keywords = []

    # Obtener las frases del texto
    phrases = [chunk.text for chunk in doc.noun_chunks if not chunk.root.is_stop]

    # Obtener los tokens relevantes para TextRank
    words = [token.text for token in doc if token.is_alpha and not token.is_stop and token.pos_ in ["NOUN", "ADJ", "VERB", "ADV"]]

    # Calcular el score de TextRank para cada palabra
    word_freq = Counter(words)
    scores = {}
    for word in word_freq:
        scores[word] = word_freq[word] / nlp(word).vector_norm

    # Ordenar las palabras por score y añadirlas a las palabras clave
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_scores[:10]:
        if word not in STOP_WORDS and word not in phrases:
            keywords.append(word)

    return keywords


['botana', 'gato']
['baby', 'minino']
['hellmanns', 'light', 'mayonesa']


En este bloque estamos definiendo tres ejemplos de texto que representan nombres de productos de supermercado. Llamamos a la función get_keywords para cada uno de estos

In [ ]:
text = "Botana Para Gato 90 Gr Atun Zizoo"
keywords = get_keywords(text)
print(keywords)

text = "Alimento Para Gato Baby Minino 1.3 Kg"
keywords = get_keywords(text)
print(keywords)

text = "Mayonesa Hellmanns Light 800 g"
keywords = get_keywords(text)
print(keywords)

Realiamos exitosamente el procesamiento de palabras claves. Queda en nosotros elegir las que vamos a utilizar.
Y claro tambien podemos automatizar creando una función que las almacene en un arreglo sin repetir como el que vamos a utilizar llamado __palabras_clave__

In [64]:
import nltk
from nltk.tokenize import word_tokenize

# Definir algunas frases de ejemplo
frases_ejemplo = []
for x in result['nombre']:
    frases_ejemplo.append(x)

In [65]:
# Definir las palabras clave
palabras_clave = ['gato', 'fabuloso', 'purina', 'cat', 'carne', 'alimento', 'mayonesa', 'heinz', 'defenseplus' , 'defensehydro' , 'gatitos', 'c/limon',
                  'hellmann', 'mccormick', 'fabuloso', 'detergente', 'limpiador', 'lavavajillas', 'lavadora', 'lavaplatos', 'lavamanos', 'light',
                  'esterilizados', 'limón', 'antibacterial', 'pasion de frutas', 'lavanda', 'nutrientes', 'mariscos', 'defenseplus', 'Whiskas', 'Minino', 'Carnita']
nombres = result['nombre']

diccionarios = []

# Crear una función que determine si dos listas de tokens tienen palabras clave en común
def tienen_palabras_clave_en_comun(nombres):
    for palabra in result['nombre']:
        palabras_clave_encontradas = []
        tokens = word_tokenize(palabra.lower())
        # print(tokens)
        for token in tokens:

            if token in palabras_clave:
                #print('Frase ->  ',  palabra,
                #      ' Palabra clave encontrada: ', token)
                palabras_clave_encontradas.append(token)

        dic = {"Producto": palabra,
               "Palabras clave encontradas": palabras_clave_encontradas}
        diccionarios.append(dic)


Ejecutamos la función para agregarlas ->

In [66]:
tienen_palabras_clave_en_comun(nombres)

In [67]:
df = pd.DataFrame(diccionarios)
df

,Producto,Palabras clave encontradas
0,Vino Tinto Chileno Gato Negro Red Blend 750ml,[gato]
1,Gato HidrÃ¡ulico de Botella Tamer 2 Toneladas,[gato]
2,Purina Cat Chow Delimix con Defenseplus Gatos ...,"[purina, cat, defenseplus]"
3,Purina Cat Chow Defenseplus Gatitos Sabor Pesc...,"[purina, cat, defenseplus, gatitos, carne]"
4,Purina Cat Chow Defense Plus para Gatos Adulto...,"[purina, cat, carne]"
...,...,...
56,Mayonesa Heinz Con Jugo De Limón 390 Gr,"[mayonesa, heinz, limón]"
57,Mayonesa La Costeña C/Limon Fsc 190gr,"[mayonesa, c/limon]"
58,Mayonesa La Costeña C/Limon Fsc 725gr,"[mayonesa, c/limon]"
59,Mayonesa Hellmanns Light 355 g Squeeze,"[mayonesa, light]"


Agregar una columna de palabras clave encontradas a su respectivo producto

In [68]:
result['palabrasClaveEncontradas'] = df['Palabras clave encontradas']
result

,precio,busqueda,nombre,url,tienda,fecha,gramaje,precioPorGramo,palabrasClaveEncontradas
0,151.00,Comida para gato,Vino Tinto Chileno Gato Negro Red Blend 750ml,https://www.chedraui.com.mx/vino-tinto-chileno...,Chedraui,2023-04-07,750.0,0.201333,[gato]
1,549.00,Comida para gato,Gato HidrÃ¡ulico de Botella Tamer 2 Toneladas,https://www.chedraui.com.mx/gato-hidraulico-de...,Chedraui,2023-04-07,2000.0,0.274500,[gato]
2,139.00,Comida para gato,Purina Cat Chow Delimix con Defenseplus Gatos ...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,1500.0,0.092667,"[purina, cat, defenseplus]"
3,120.00,Comida para gato,Purina Cat Chow Defenseplus Gatitos Sabor Pesc...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,1500.0,0.080000,"[purina, cat, defenseplus, gatitos, carne]"
4,259.00,Comida para gato,Purina Cat Chow Defense Plus para Gatos Adulto...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,3000.0,0.086333,"[purina, cat, carne]"
...,...,...,...,...,...,...,...,...,...
56,54.0,Mayonesa,Mayonesa Heinz Con Jugo De Limón 390 Gr,https://www.soriana.com/mayonesa-heinz-con-jug...,Soriana,2023-04-07,390.0,0.138462,"[mayonesa, heinz, limón]"
57,28.8,Mayonesa,Mayonesa La Costeña C/Limon Fsc 190gr,https://www.soriana.com/mayonesa-la-costena-c-...,Soriana,2023-04-07,190.0,0.151579,"[mayonesa, c/limon]"
58,83.2,Mayonesa,Mayonesa La Costeña C/Limon Fsc 725gr,https://www.soriana.com/mayonesa-la-costena-c-...,Soriana,2023-04-07,725.0,0.114759,"[mayonesa, c/limon]"
59,45.9,Mayonesa,Mayonesa Hellmanns Light 355 g Squeeze,https://www.soriana.com/mayonesa-hellmanns-lig...,Soriana,2023-04-07,355.0,0.129296,"[mayonesa, light]"


Guardar en mongoDB el dataframe result

In [69]:
client = MongoClient('localhost', 27017)
db = client['productos']
coleccion = db['productos']
result_dict = result.to_dict(orient='records')
coleccion.insert_many(result_dict)
print('Grupo', result_dict[0], 'almacenado en MongoDB.')



Grupo {'precio': '151.00', 'busqueda': 'Comida para gato', 'nombre': 'Vino Tinto Chileno Gato Negro Red Blend 750ml', 'url': 'https://www.chedraui.com.mx/vino-tinto-chileno-gato-negro-red-blend-750ml-3716120/p', 'tienda': 'Chedraui', 'fecha': '2023-04-07', 'gramaje': 750.0, 'precioPorGramo': 0.20133333333333334, 'palabrasClaveEncontradas': ['gato'], '_id': ObjectId('643073257d070c556e5298c8')} almacenado en MongoDB.


In [70]:
# Leer los datos de MongoDB
cursor = coleccion.find()
# Convertir el cursor a una lista de documentos
documentos = list(cursor)
# Convertir la lista de documentos a un DataFrame de Pandas
df = pd.DataFrame(documentos)

In [71]:
#Seleccionar todos los productos que se llamen Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg y agrupar por fecha
df[df['nombre'] == 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg']

,_id,nombre,busqueda,precio,url,tienda,fecha,gramaje,precioPorGramo,palabrasClaveEncontradas
1,64187e8ca190e8d29890074b,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,277.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-03-20,NaN,NaN,NaN
50,64284e4ac4a2dc63bf488aa0,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,NaN,NaN,NaN
107,64286681c4a2dc63bf488ada,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,NaN,NaN,NaN
164,6428c54c3fbc3434bd227675,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,3000.0,0.086333,NaN
187,642f28f0ec5b8fa3a91e2bd1,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-06,NaN,NaN,NaN
247,642f39a1ec5b8fa3a91e2c0e,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-06,NaN,NaN,NaN
305,64307166e522cc4aa39aac2a,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,NaN,NaN,NaN
332,643073257d070c556e5298cc,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,3000.0,0.086333,"[purina, cat, carne]"
